In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import statsmodels.api as sm
import seaborn as sns

In [ ]:
# df = pd.read_excel('/Users/bryan/Documents/SeqDataSets/200313/200313s09_100_full.xls')
# df = pd.read_csv("/Users/bryan/Studio/tools/mix_10.csv")
# df = pd.read_csv('/Users/bryan/Documents/SeqDataSets/12mix/12mix.csv')
df = pd.read_excel("/Users/bryan/Studio/seq_bank/19.xlsx")
# df = pd.read_csv("/Users/bryan/Studio/seq/data/samples/19.csv")
# df = pd.read_excel('/Users/bryan/Studio/seq_bank/data/merge/mix_19_20_21_201_202_203_204_205_206_207.xlsx')
# df = pd.read_csv('/Users/bryan/Studio/seq/data/samples/201.csv')
# df = pd.read_csv('/Users/bryan/Documents/SeqDataSets/signal_to_noise_20/NZ_180709_s06_0610_20_k.xls')
# df = pd.read_csv('/Users/bryan/Documents/SeqDataSets/signal_to_noise_20/MIX/NZ_180709_S05_0610_20_k.csv')
# df = pd.read_excel('/Users/bryan/Documents/SeqDataSets/signal_to_noise_20/MIX/NZ_180709_S05_0610_20.xls')
# df = pd.read_csv('/Users/bryan/Studio/seq/data/merges/df_19_20_21_201_202_203_204_205_206_207.csv')
# df = pd.read_excel('/Users/bryan/Studio/seq_bank/694.2397_500_20200624112302.xlsx')
df.shape

In [ ]:
df[df.Mass == 6954.9496]

In [ ]:
df3p = pd.read_csv('/Users/bryan/Studio/seq2/3p.csv')
df5p = pd.read_csv('/Users/bryan/Studio/seq2/5p.csv')

In [ ]:
plt.scatter(df3p['Mass'], df3p['RT'])

In [ ]:
plt.scatter(df5p['Mass'], df5p['RT'])

In [ ]:
# TAG = 826.3184
TAG = 694.2397
def init_df(df, tag=694.2397):
    df = df.append({'Mass': 18.0106, 'RT': 0.66, 'Vol': 10000}, ignore_index=True)
    df = df.append({'Mass': 347.0631, 'RT': 0.99, 'Vol': 10000}, ignore_index=True)
    df = df.append({'Mass': 323.0519, 'RT': 0.99, 'Vol': 10000}, ignore_index=True)
    df = df.append({'Mass': 363.058, 'RT': 0.99, 'Vol': 10000}, ignore_index=True)
    df = df.append({'Mass': 324.0359, 'RT': 0.99, 'Vol': 10000}, ignore_index=True)
    # df = df.append({'Mass': 826.3184, 'RT': 1.66, 'Vol': 20000}, ignore_index=True)
    df = df.append({'Mass': tag, 'RT': 1.66, 'Vol': 20000}, ignore_index=True)
    df.shape
    return df

In [ ]:
plt.figure(figsize=(12, 9))
plt.scatter(df['Mass'], df['RT'])

In [ ]:
def base_calling_random(df):
    df = df.sort_index()
    mass_x = np.array(df.Mass)
    mass_y = np.array(df.Mass)
    print("mass_x {} mass_y {}".format(mass_x.shape, mass_y.shape))

    bases = [305.0413, 329.0525, 345.0474, 306.0253, 359.0631, 308.041, 319.0569, 320.041, 373.0787, 343.0682, 570.1475, 212.0086, 557.2251, 688.1156, 625.0844]
    base_dict = {'C': 305.0413, 'A': 329.0525, 'G': 345.0474, 'U': 306.0253, 'g': 359.0631, 'D': 308.041, 'c': 319.0569, 'T': 320.041, 'G^': 373.0787, 'a': 343.0682, 'y': 212.0086, 'Y': 570.1475, 'P': 557.2251, 'x': 688.1156, 'z': 625.0844}
    pairs = list()
    idxs = list()
    PPM = 10
    for k in base_dict.keys():
        base_mass = base_dict.get(k)
        ppm_matrix = np.abs((mass_x[:, np.newaxis] - mass_y - base_mass) * 1E6 / (mass_y + base_mass))
        ppm_df = pd.DataFrame(ppm_matrix)
        crosstalk = ppm_df[ppm_df < 10]
        idx_pairs = list(crosstalk[crosstalk.notnull()].stack().index)
        if idx_pairs:
            idx_pairs = [(*pair, k) for pair in idx_pairs] # append base name into idx_pairs
            pairs.extend(idx_pairs)
        df3_idxs = [pair[0] for pair in idx_pairs]
        df5_idxs = [pair[1] for pair in idx_pairs]
        df3_idxs = list(set(df3_idxs))
        df5_idxs = list(set(df5_idxs))
        print("df3_idxs {} df5_idxs {}".format(len(df3_idxs), len(df5_idxs)))
        idxs.extend(df3_idxs)
        idxs.extend(df5_idxs)
    
    mass_pairs = [(df.Mass.iloc[p[0]], df.Mass.iloc[p[1]], p[2]) for p in pairs]
    idxs = list(set(idxs))
    plt.figure(figsize=(16, 12))
    df_base_calling = df.iloc[idxs]
    print(df_base_calling.shape)
    return df_base_calling, mass_pairs

In [ ]:
def get_end5p_points(df5p, sum_value):
    end_5p = sum_value - 80 - TAG
    masses = df5p.Mass
    print(end_5p)
    BIAS = 0.3
    end5p_idxs = masses[(masses>=end_5p-BIAS) & (masses<=end_5p+BIAS)].index
    print("end5p_idxs {}".format(end5p_idxs))
    
    endpoints = df5p.loc[end5p_idxs]
    endpoints = endpoints.sort_values('Vol', ascending=False)
    return endpoints.iloc[0]

# df5_idxs.extend(list(end5p_idxs))

In [ ]:
def computational_data_seperation(df3p, df5p, full_mass, ignore_endpoints=False):
    df3p_mass_np = np.array(df3p['Mass'])
    df5p_mass_np = np.array(df5p['Mass'])
    mass_sum_np = df3p_mass_np[:, np.newaxis] + df5p_mass_np
    mass_sum_1 = np.round(mass_sum_np, 1)
    mass_sum_df = pd.DataFrame(mass_sum_np)
    mass_sum_df1 = pd.DataFrame(mass_sum_1)

    sum_value = round(full_mass + 18.0106, 1)

    tmp = mass_sum_df1[(mass_sum_df1 >= sum_value-0.1) & (mass_sum_df1 <= sum_value+0.1)]
    final_idx_pairs = list(tmp[tmp.notnull()].stack().index)
    df3_idxs = [pair[0] for pair in final_idx_pairs]
    df5_idxs = [pair[1] for pair in final_idx_pairs]
    df3_idxs = list(set(df3_idxs))
    df5_idxs = list(set(df5_idxs))
    print("df3_idxs {} df5_idxs {}".format(len(df3_idxs), len(df5_idxs)))

    df3p_selected = df3p.iloc[df3_idxs]
    df5p_selected = df5p.iloc[df5_idxs]
    
    if not ignore_endpoints:
        df5p_endpoints = get_end5p_points(df5p, full_mass + 18.0106)
        print("df5p_endpoints {}".format(df5p_endpoints[['Mass', 'RT', 'Vol']]))

        df3p_selected = df3p_selected.append(df5p_endpoints)
        df5p_selected = df5p_selected.append(df5p_endpoints)
        
    return df3p_selected, df5p_selected

In [ ]:
def label_unlabel_comparison(df3p, df5p, diff=694.2397 + 61.9557):
    df3p_mass_np = np.array(df3p['Mass'])
    df5p_mass_np = np.array(df5p['Mass'])
    mass_sum_np = df3p_mass_np[:, np.newaxis] - df5p_mass_np
    mass_sum_1 = np.round(mass_sum_np, 1)
    mass_sum_df = pd.DataFrame(mass_sum_np)
    mass_sum_df1 = pd.DataFrame(mass_sum_1)

    comparee = round(diff, 1)

    tmp = mass_sum_df1[(mass_sum_df1 >= comparee-0.1) & (mass_sum_df1 <= comparee+0.1)]
    final_idx_pairs = list(tmp[tmp.notnull()].stack().index)
    df3_idxs = [pair[0] for pair in final_idx_pairs]
    df5_idxs = [pair[1] for pair in final_idx_pairs]
    df3_idxs = list(set(df3_idxs))
    df5_idxs = list(set(df5_idxs))
    print("df3_idxs {} df5_idxs {}".format(len(df3_idxs), len(df5_idxs)))

    df3p_selected = df3p.iloc[df3_idxs]
    df5p_selected = df5p.iloc[df5_idxs]
    return df3p_selected, df5p_selected

In [ ]:
def plot_zones(df3p, df5p):
    plt.figure(figsize=(12, 9))
    sns.regplot(df3p.Mass, df3p.RT)
    sns.regplot(df5p.Mass, df5p.RT)

def plot_zone(df):
    plt.figure(figsize=(12, 9))
    plt.scatter(df.Mass, df.RT)

In [ ]:
df3p = df
df5p = df
df3p.shape, df5p.shape

In [ ]:
src_to_transfer = '/Users/bryan/Downloads/gradientyeastsample4.xlsx'
df = pd.read_excel(src_to_transfer)
df.rename(columns={'Monoisotopic Mass': 'Mass', 'Sum Intensity': 'Vol', 'Apex RT': 'RT'}, inplace=True)
df.to_excel(src_to_transfer)

In [ ]:
df = pd.read_excel('/Users/bryan/Downloads/oligosdegradsample2.xlsx')
df.shape

In [ ]:
df_big = df[df.Mass>22000].sort_values('Vol', ascending=False)[:10][['Mass', 'Vol', 'RT']]
df_big

In [ ]:
# full_mass = 24581.37038 # phe
# full_mass = 24252.354538 # phe2
# full_mass = 24233.17434 # Asp
# full_mass = 24287.342597 # unknow
full_mass = df_big.iloc[1].Mass
print(full_mass)
df_ms, _ = computational_data_seperation(df, df, full_mass=full_mass, ignore_endpoints=True)
df_ms.shape

In [ ]:
df_ms_bcr, mass_pairs = base_calling_random(df_ms)
df_ms_bcr.shape, len(mass_pairs)

In [ ]:
df_ms_bcr.info()

In [ ]:
%run ../modules/utils.ipynb
plot_basecalling2(df_ms_bcr, mass_pairs)

In [ ]:
plt.figure(figsize=(16, 12))
plt.scatter(df_ms.Mass, df_ms.RT)
mass1 = [t[0] for t in mass_pairs]
mass2 = [t[1] for t in mass_pairs]

for t in mass_pairs:
    df_pair = df_ms[df_ms.Mass.isin(t)]
#     print("p1 {} p2 {}".format(p1.Mass.values[0], p2.Mass))
    plt.plot(df_pair.Mass, df_pair.RT, 'black')
    plt.annotate(s=t[2], size=15, xy=(df_pair.Mass[:1], df_pair.RT[:1]), textcoords="offset points", xytext=(0,10), ha='center')

plt.title('tRNA-phe')
plt.show()

In [ ]:
df = df.sort_values('Vol', ascending=False)

In [ ]:
df.iloc[:5][['Mass', 'Vol']]

In [ ]:
df = pd.read_excel('/Users/bryan/Studio/seq_bank/694.2397_500_20200624112302.xlsx')
df.shape

In [ ]:
df1 = pd.read_excel('/Users/bryan/Documents/SeqDataSets/PaperData/ACS/100918s06.xls')
df2 = pd.read_excel('/Users/bryan/Downloads/oligosdegradsample2.xlsx')
df1.shape, df2.shape

In [ ]:
df1.columns, df2.columns

In [ ]:
df_phe = pd.concat([df1[['Mass', 'RT', 'Vol']], df2[['Mass', 'RT', 'Vol']]])

In [ ]:
df_ms.to_excel('data/asp_merge.xlsx')

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
def computational_data_seperation(df3p, df5p, full_mass, ignore_endpoints=False):
    df3p_mass_np = np.array(df3p['Mass'])
    df5p_mass_np = np.array(df5p['Mass'])
    mass_sum_np = df3p_mass_np[:, np.newaxis] + df5p_mass_np
    mass_sum_1 = np.round(mass_sum_np, 1)
    mass_sum_df = pd.DataFrame(mass_sum_np)
    mass_sum_df1 = pd.DataFrame(mass_sum_1)

    sum_value = round(full_mass + 18.0106, 1)

    bias = 0.2
    tmp = mass_sum_df1[(mass_sum_df1 >= sum_value-bias) & (mass_sum_df1 <= sum_value+bias)]
    final_idx_pairs = list(tmp[tmp.notnull()].stack().index)
    df3_idxs = [pair[0] for pair in final_idx_pairs]
    df5_idxs = [pair[1] for pair in final_idx_pairs]
    df3_idxs = list(set(df3_idxs))
    df5_idxs = list(set(df5_idxs))
    print("df3_idxs {} df5_idxs {}".format(len(df3_idxs), len(df5_idxs)))

    df3p_selected = df3p.iloc[df3_idxs]
    df5p_selected = df5p.iloc[df5_idxs]
    
    if not ignore_endpoints:
        df5p_endpoints = get_end5p_points(df5p, full_mass + 18.0106)
        print("df5p_endpoints {}".format(df5p_endpoints[['Mass', 'RT', 'Vol']]))

        df3p_selected = df3p_selected.append(df5p_endpoints)
        df5p_selected = df5p_selected.append(df5p_endpoints)
        
    return df3p_selected, df5p_selected

In [ ]:
plot_zone(df3p)

In [ ]:
#24581.37038
full_mass = 24581.37038
df_ms, _ = computational_data_seperation(df2, df2, full_mass=full_mass, ignore_endpoints=True)

In [ ]:
df2.columns

In [ ]:
plt.figure(figsize=(15,11))
plt.scatter(df2[df2.Mass > 22000].Mass, df2[df2.Mass > 22000]['Sum Intensity'])

In [ ]:
plt.figure(figsize=(15, 11))
plt.scatter(df_ms.Mass, df_ms.RT)

In [ ]:
df3p.to_excel('data/tmp_mix500.xlsx')

In [ ]:
full_masses = {'19': 6781.0752, '20': 7079.0804, '21': 7522.1083, '201': 7088.0843, '202': 6985.0440, 
               '203': 7073.0746, '204': 6954.9496, '205': 7110.0903, '206': 7151.1211, '207': 7193.0484}
for k in full_masses.keys():
    full_mass = full_masses.get(k)
    df_seq, _ = computational_data_seperation(df3p, df5p, full_mass=full_mass)
    df_seq.to_excel('data/merge/mix_{}_{}.xlsx'.format(len(full_masses), k))
    df_seq.shape
#     plot_zone(df_seq)

In [ ]:
# full_mass = 6847.0787
# full_mass = 6929.0318
full_mass = 6781.0752 #19
# full_mass = 7079.0804 #20
# full_mass = 7522.1083 #21
# full_mass = 7088.0843 #201
# full_mass = 6985.0440 #202
# full_mass = 7073.0746 #203
# full_mass = 6954.9496 #6954.9868 #204
# full_mass = 7110.0903 #205
# full_mass = 7151.1211 #206
# full_mass = 7193.0484 #207


In [ ]:
df = pd.read_csv('/Users/bryan/Documents/SeqDataSets/2mix/NZ_180521s07_19_20.csv')
print(df.shape)
df = init_df(df)
df3p = df5p = df
df_seq, _ = computational_data_seperation(df3p, df5p, full_mass=full_mass)
# df_seq.to_excel('data/merge_pure/mix_1_19.xlsx')
df_seq.shape
plot_zone(df_seq)

In [ ]:
df_3p, df_5p = computational_data_seperation(df3p, df5p, full_mass=full_mass)
df_3p.shape, df_5p.shape

In [ ]:
plot_zones(df_3p, df_5p)

In [ ]:
df_seq = df_seq[df_seq.RT < 15]
z = np.polyfit(df_seq.Mass, df_seq.RT, 1)
def f(x):
    return z[0]*x + z[1]
plt.scatter(df_seq.Mass, df_seq.RT)
plt.plot(df_seq.Mass, f(df_seq.Mass))

In [ ]:
# df_seq_3p = df_seq[df_seq.RT > f(df_seq.Mass)]
# df_seq_5p = df_seq[df_seq.RT <= f(df_seq.Mass)]
df_seq_3p.shape, df_seq_5p.shape

In [ ]:
plot_zones(df_seq_3p, df_seq_5p)

In [ ]:
df_label, df_unlabel = label_unlabel_comparison(df3p, df5p)

In [ ]:
plot_zones(df_label, df_unlabel)

In [ ]:
df_common_3p = pd.merge(df_label, df_seq_3p, how='inner')
df_common_3p.shape

In [ ]:
plot_zone(df_common_3p)

In [ ]:
df_common_3p_bc = base_calling_random(df_common_3p)
df_common_3p_bc.shape

In [ ]:
plot_zone(df_common_3p_bc)

In [ ]:
df_common_3p_lbl, df_unlbl = label_unlabel_comparison(df_common_3p, df_unlabel)

In [ ]:
plot_zone(df_unlbl)

In [ ]:
df_unlbl_bc = base_calling_random(df_unlbl)
df_unlbl_bc.shape

In [ ]:
plot_zones(df_common_3p_lbl, df_unlbl_bc)

In [ ]:
df_common_3p_tmp, df_seq_5p_tmp = computational_data_seperation(df_common_3p, df_seq_5p)

In [ ]:
plot_zones(df_common_3p_tmp, df_seq_5p_tmp)

In [ ]:
df_common_3p_tmp[['Mass', 'RT', 'Vol', 'Quality Score']]

In [ ]:
df_common_3p_lbl.to_excel('data/19_3p.xls')
df_seq_5p_tmp.to_excel('data/19_5p.xls')

In [ ]:
# df = df.sort_values(by='Vol', ascending=False)
df = df.sort_index()
df.to_csv('data/5p.csv')

In [ ]:
df3p_mass_np = np.array(df3p['Mass'])
df5p_mass_np = np.array(df5p['Mass'])
mass_sum_np = df3p_mass_np[:, np.newaxis] + df5p_mass_np

In [ ]:
mass_sum_0 = np.round(mass_sum_np, 0)
mass_sum_1 = np.round(mass_sum_np, 1)
mass_sum_2 = np.round(mass_sum_np, 2)
mass_sum_3 = np.round(mass_sum_np, 3)

In [ ]:
# seq_mass = 7281.1982
# seq_mass = 7079.0861
seq_mass = 6781.0487 #6781.0763
# seq_mass = 7522.1079
# seq_mass = 7088.0775
# seq_mass = 7073.0735
# seq_mass = 6954.9831
# seq_mass = 7110.0883
# seq_mass = 7151.113
# seq_mass = 7193.0523
# seq_mass = 6781.0763
# seq_mass = 6985.9879
sum_value = round(seq_mass + 18.0106, 1)
mass_sum_1[(mass_sum_1 >= sum_value-0.2) & (mass_sum_1 <=sum_value+0.2)]

In [ ]:
end_5p = sum_value - 80 - TAG
masses = df5p.Mass
print(end_5p)
BIAS = 0.1
end5p_idxs = masses[(masses>=end_5p-BIAS) & (masses<=end_5p+BIAS)].index
df5p.loc[end5p_idxs]

In [ ]:
# tmp = mass_sum_1[((mass_sum_1 >= sum_value-0.1) & (mass_sum_1 <= sum_value+0.1)) | ((mass_sum_1 >= sum_value-80-0.1) & (mass_sum_1 <= sum_value-80+0.1))]
tmp = mass_sum_1[(mass_sum_1 >= sum_value-0.1) & (mass_sum_1 <= sum_value+0.1)]
len(tmp)

In [ ]:
mass_sum_df = pd.DataFrame(mass_sum_np)
mass_sum_df1 = pd.DataFrame(mass_sum_1)

# tmp = mass_sum_df1[((mass_sum_df1 >= sum_value-0.1) & (mass_sum_df1 <= sum_value+0.1)) | ((mass_sum_df1 >= sum_value-80-0.1) & (mass_sum_df1 <= sum_value-80+0.1))]
tmp = mass_sum_df1[(mass_sum_df1 >= sum_value-0.1) & (mass_sum_df1 <= sum_value+0.1)]
final_idx_pairs = list(tmp[tmp.notnull()].stack().index)
df3_idxs = [pair[0] for pair in final_idx_pairs]
df5_idxs = [pair[1] for pair in final_idx_pairs]
# df5_idxs += df3_idxs
# print("df3 idxs {} \ndf5 idxs {}".format(df3_idxs, df5_idxs))

# df3p_mass_np[df3_idxs]
# df5p_mass_np[df5_idxs]
print("df3_idxs {} df5_idxs {}".format(len(df3_idxs), len(df5_idxs)))

In [ ]:
df3_idxs_tmp = list(set(df3_idxs))
len(df3_idxs_tmp)

In [ ]:
df3_idxs = list(set(df3_idxs))
df5_idxs = list(set(df5_idxs))
df5_idxs.extend(list(end5p_idxs))
print(len(df5_idxs))

In [ ]:
plt.figure(figsize=(12, 9))
df3p_selected = df3p.iloc[df3_idxs]
df5p_selected = df5p.iloc[df5_idxs]
print(df5p_selected.shape, df3p_selected.shape)
df5p_selected = df5p_selected.sort_values('Mass')
plt.scatter(df3p_selected['Mass'], df3p_selected['RT'], color='r')
plt.scatter(df5p_selected['Mass'], df5p_selected['RT'])

In [ ]:
df3p_selected = df3p_selected.sort_values('Vol', ascending=False)

In [ ]:
df3p_top = df3p_selected.iloc[:200]
plt.figure(figsize=(12, 9))
plt.scatter(df3p_top.Mass, df3p_top.RT)

In [ ]:
df3p_basecalling = base_calling_random(df3p_selected)

In [ ]:
df5p_basecalling = base_calling_random(df5p_selected)

In [ ]:
df3p_selected.to_excel('data/first.xlsx')

In [ ]:
df3p_cds, df5p_cds = computational_data_seperation(df3p_basecalling, df5p_basecalling)

In [ ]:
plot_zones(df3p_cds, df5p_cds)

In [ ]:
df3p_cds = base_calling_random(df3p_cds)
df5p_cds = base_calling_random(df5p_cds)